In [17]:

from collections import namedtuple
from PIL import Image, ImageDraw, ImageFont
from google.cloud import vision
from enum import Enum
%matplotlib inline
import matplotlib.pyplot as plt
import os
import json
from google.protobuf.json_format import MessageToJson
from google.protobuf import json_format
from pdf2image import convert_from_path
import PyPDF2


import sys
sys.path.append("..")
from models.bounding_box import FeatureType, Point, BoundingBox
from utils.cv_preprocess import draw_boxes, group_bounding_boxes, merge_box_groups

from utils.file_utils import create_dir, prepare_image_local, prepare_image_web, load_counter, save_counter, save_json

from utils.nlp_preprocess import load_spacy_models, is_english, is_chinese, split_dish_info, is_word_relevant

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../menu-ocr-390814-027d51b70720.json'


In [18]:
filein = '../downloads/Ma-La-Menu_07-2023.pdf'
base_name = os.path.basename(filein)
file_name_without_extension = os.path.splitext(base_name)[0]


raw_ocr_directory = '../output/raw_ocr/'
raw_ocr_filename = file_name_without_extension + "_raw_annotation.json"
raw_ocr_path = os.path.join(raw_ocr_directory, raw_ocr_filename)

preprocessed_ocr_directory = '../output/prep_ocr/'
preprocessed_ocr_filename = file_name_without_extension + "_prep_ocr.json"
preprocessed_ocr_path = os.path.join(preprocessed_ocr_directory, preprocessed_ocr_filename)

menu_image_directory = '../downloaded_menu/unclassified'
create_dir(menu_image_directory)

def list_pdf_files(directory):
    """List all PDF files in the specified directory."""
    return [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.pdf')]

def convert_pdf_to_images(pdf_path, output_folder, dpi=300):
    # Open the PDF file
    doc = fitz.open(pdf_path)
    base_name = os.path.basename(pdf_path)
    file_name_without_extension = os.path.splitext(base_name)[0]

    # Set zoom factor for increased resolution
    zoom_x = zoom_y = dpi / 72  # 72 is the default DPI in PyMuPDF
    mat = fitz.Matrix(zoom_x, zoom_y)

    # Iterate through each page
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)  # number of page
        pix = page.get_pixmap(matrix=mat)  # render page to an image with increased resolution
        export_filename = f"{file_name_without_extension}_page_{page_num}.png"
        output_image_path = os.path.join(output_folder, export_filename)
        pix.save(output_image_path)  # save image as a PNG
        print(f"Page {page_num} has been processed and saved as {export_filename}")

    doc.close()


# Update here to process all PDF files in the directory
pdf_directory = '../downloaded_menu/edinburgh'  # Update this path to the directory containing the PDFs
pdf_files = list_pdf_files(pdf_directory)

for pdf_file in pdf_files:
    try:
        convert_pdf_to_images(pdf_file, menu_image_directory)
        print(f"Processed all pages of {pdf_file}")
    except Exception as e:
        print(f"An error occurred while processing {pdf_file}: {str(e)}")

Page 0 has been processed and saved as Rendezvous_sit-in_08-23_Draft_page_0.png
Page 1 has been processed and saved as Rendezvous_sit-in_08-23_Draft_page_1.png
Page 2 has been processed and saved as Rendezvous_sit-in_08-23_Draft_page_2.png
Page 3 has been processed and saved as Rendezvous_sit-in_08-23_Draft_page_3.png
Page 4 has been processed and saved as Rendezvous_sit-in_08-23_Draft_page_4.png
Page 5 has been processed and saved as Rendezvous_sit-in_08-23_Draft_page_5.png
Page 6 has been processed and saved as Rendezvous_sit-in_08-23_Draft_page_6.png
Page 7 has been processed and saved as Rendezvous_sit-in_08-23_Draft_page_7.png
Page 8 has been processed and saved as Rendezvous_sit-in_08-23_Draft_page_8.png
Page 9 has been processed and saved as Rendezvous_sit-in_08-23_Draft_page_9.png
Processed all pages of ../downloaded_menu/edinburgh\Rendezvous_sit-in_08-23_Draft.pdf
Page 0 has been processed and saved as The-Golden-Dragon-Restaurant-Menu_page_0.png
Page 1 has been processed and 

In [19]:
import fitz  # PyMuPDF
import os

def convert_pdf_to_images(pdf_path, output_folder, dpi=300):
    # Open the PDF file
    doc = fitz.open(pdf_path)

    # Set zoom factor for increased resolution
    zoom_x = zoom_y = dpi / 72  # 72 is the default DPI in PyMuPDF
    mat = fitz.Matrix(zoom_x, zoom_y)

    # Iterate through each page
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)  # number of page
        pix = page.get_pixmap(matrix=mat)  # render page to an image with increased resolution
        output_image_path = os.path.join(output_folder, f"page_{page_num + 1}.png")
        pix.save(output_image_path)  # save image as a PNG

    doc.close()

# Example usage
pdf_file =  '../downloaded_menu/manchester/2023-a-la-carte-menu-264.pdf'  # Replace with your PDF file path
output_directory = '../downloaded_menu/unclassified'  # Replace with your desired output directory
convert_pdf_to_images(pdf_file, output_directory)
